## USING SYSTEMS CAMERA

In [2]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from time import time
import cvzone
from screeninfo import get_monitors
import warnings
warnings.filterwarnings('ignore')

screen_width = get_monitors()[0].width
screen_height = get_monitors()[0].height
cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Image", screen_width, screen_height)

# Start Video Capture
cap = cv2.VideoCapture(0)  # Adjust the camera
cap.set(3, screen_width)
cap.set(4, screen_height)

detector = HandDetector(detectionCon=0.8)

# Keyboard Layout
keys = [
    ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "<"],
    ["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
    ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";"],
    ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/"],
    ["Space", "Enter"]
]
finalText = ""

class Button:
    def __init__(self, pos, text, size=[60, 60]):  # Adjusted size of the buttons
        self.pos = pos
        self.size = size
        self.text = text

def drawAll(img, buttonList):
    for button in buttonList:
        x, y = button.pos
        w, h = button.size
        cvzone.cornerRect(img, (button.pos[0], button.pos[1], button.size[0], button.size[1]), 10, rt=4)
        cv2.rectangle(img, button.pos, (x + w, y + h), (255, 255, 255), cv2.FILLED)
        cv2.putText(img, button.text, (x + 5, y + 30), cv2.FONT_HERSHEY_PLAIN, 2, (150, 150, 150), 2)
    return img

# Create button list
buttonList = []
for i in range(len(keys)):
    for j, key in enumerate(keys[i]):
        if key == "Space":
            buttonList.append(Button([450, 80 * i + 90], key, size=[160, 60]))  # Adjusted size for space button
        elif key == "Enter":
            buttonList.append(Button([650, 80 * i + 90], key, size=[160, 60]))  # Adjusted position for enter button
        else:
            buttonList.append(Button([80 * j + 220, 100 * i + 10], key))

# Variables to manage debounce and stability effect
lastKeyPressed = None
lastPressTime = 0
pressCooldown = 0.5  # Time in seconds
stableTime = 0.2  # Time in seconds
startStableTime = None
currentButton = None

while True:
    success, img = cap.read()
    if not success:
        print("Failed to capture image")
        break
    
    hands, img = detector.findHands(img, draw=True, flipType=True)
    img = drawAll(img, buttonList)

    if hands:
        lmListHand = hands[0]['lmList']
        for button in buttonList:
            x, y = button.pos
            w, h = button.size

            if x < lmListHand[8][0] < x + w and y < lmListHand[8][1] < y + h:
                if currentButton != button.text:
                    startStableTime = time()
                    currentButton = button.text

                if currentButton == button.text:
                    if startStableTime is not None and time() - startStableTime > stableTime:
                        cv2.rectangle(img, (x - 5, y - 5), (x + w + 5, y + h + 5), (130, 80, 70), cv2.FILLED)
                        cv2.putText(img, button.text, (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 2, (200, 200,200), 2)

                        l, _, _ = detector.findDistance((lmListHand[8][0], lmListHand[8][1]), 
                                                        (lmListHand[12][0], lmListHand[12][1]), img)

                        currentTime = time()
                        if l < 30 and (lastKeyPressed != button.text or (currentTime - lastPressTime) > pressCooldown):
                            #print(f"Key Pressed: {button.text}")
                            cv2.rectangle(img, button.pos, (x + w, y + h), (230,50,230), cv2.FILLED)
                            cv2.putText(img, button.text, (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
                            if button.text == "<":  # Handle backspace
                                finalText = finalText[:-1]
                            elif button.text == "Space":  # Handle space
                                finalText += " "
                            elif button.text == "Enter":  # Handle enter
                                print(f"{finalText}")
                                finalText = ""  # Clear the text after printing
                            else:
                                finalText += button.text
                            lastKeyPressed = button.text
                            lastPressTime = currentTime
                            startStableTime = None  # Reset stable time after a key press

    else:
        currentButton = None  # Reset currentButton when no hands are detected

    # Draw text box
    #cv2.rectangle(img, (100, 550), (1150, 600), (175, 0, 175), cv2.FILLED)  # Adjusted text box position and size
    cv2.putText(img, finalText, (50, 600), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


HI SAM


## COMBINING KEYBOARD AND GESTURE REGOZNITION

In [7]:
import os
import joblib
import cv2
import pandas as pd
import numpy as np
from cvzone.HandTrackingModule import HandDetector
import math
import time
import cvzone
from screeninfo import get_monitors
import warnings

warnings.filterwarnings('ignore')

# Initialize CV2 window
screen_width = get_monitors()[0].width
screen_height = get_monitors()[0].height
cv2.namedWindow("Hand Gesture Recognition and Keyboard", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Hand Gesture Recognition and Keyboard", screen_width, screen_height)

# Start Video Capture for Hand Gesture Recognition
cap_gesture = cv2.VideoCapture(0)
cap_gesture.set(3, screen_width)
cap_gesture.set(4, screen_height)

# Start Video Capture for Keyboard Input
cap_keyboard = cv2.VideoCapture(0)  # Adjust the camera
cap_keyboard.set(3, screen_width)
cap_keyboard.set(4, screen_height)

def calculate_angle(A, B, C):
    """Calculate the angle between points A, B, and C.
    
    The points are given as (x, y) coordinates.
    """
    BA = (A[0] - B[0], A[1] - B[1])
    BC = (C[0] - B[0], C[1] - B[1])
    magnitude_BA = math.sqrt(BA[0] ** 2 + BA[1] ** 2)
    magnitude_BC = math.sqrt(BC[0] ** 2 + BC[1] ** 2)
    
    if magnitude_BA == 0 or magnitude_BC == 0:
        return 0.0

    cosine_angle = (BA[0] * BC[0] + BA[1] * BC[1]) / (magnitude_BA * magnitude_BC)
    cosine_angle = max(-1.0, min(1.0, cosine_angle))
    angle = math.degrees(math.acos(cosine_angle))
    return angle

# Load the trained model for gesture recognition
model_path = r'D:\SpectoV\Hand_Gesture_Recognition\research\addons\models\RFC_MOEL_1_A_Z.pkl'
clf = joblib.load(model_path)

# Initialize the HandDetector
detector = HandDetector(staticMode=False, maxHands=2, modelComplexity=1, detectionCon=0.5, minTrackCon=0.5)

# Define the list of key points (landmarks)
num_landmarks = 21

# Dictionary mapping numerical labels to text keys
label_map = {
    0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
    10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S',
    19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'
}

# Function to calculate distances and angles between key points
def calculate_features(coords):
    distances = []
    for i in range(len(coords)):
        for j in range(len(coords)):
            if i < j:
                distance = np.linalg.norm(np.array(coords[i]) - np.array(coords[j]))
                distances.append(distance)
    return distances

# Define the confidence threshold for gesture recognition
confidence_threshold = 0.40

# Variables to manage gesture recognition
sentence = ""
previous_gesture = ""
frame_count = 0
no_hand_frames = 0
recognition_count = 0

# Variables to manage keyboard input
keys = [
    ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "<"],
    ["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
    ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";"],
    ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/"],
    ["Space", "Enter"]
]
finalText = ""

class Button:
    def __init__(self, pos, text, size=[60, 60]):  # Adjusted size of the buttons
        self.pos = pos
        self.size = size
        self.text = text

def drawAll(img, buttonList):
    for button in buttonList:
        x, y = button.pos
        w, h = button.size
        cvzone.cornerRect(img, (button.pos[0], button.pos[1], button.size[0], button.size[1]), 10, rt=4)
        cv2.rectangle(img, button.pos, (x + w, y + h), (255, 255, 255), cv2.FILLED)
        cv2.putText(img, button.text, (x + 5, y + 30), cv2.FONT_HERSHEY_PLAIN, 2, (150, 150, 150), 2)
    return img

# Create button list for keyboard interface
buttonList = []
for i in range(len(keys)):
    for j, key in enumerate(keys[i]):
        if key == "Space":
            buttonList.append(Button([450, 80 * i + 90], key, size=[160, 60]))  # Adjusted size for space button
        elif key == "Enter":
            buttonList.append(Button([650, 80 * i + 90], key, size=[160, 60]))  # Adjusted position for enter button
        else:
            buttonList.append(Button([80 * j + 220, 100 * i + 10], key))

# Variables to manage debounce and stability effect for keyboard input
lastKeyPressed = None
lastPressTime = 0
pressCooldown = 0.5  # Time in seconds
stableTime = 0.2  # Time in seconds
startStableTime = None
currentButton = None

# Define mode switching key for keyboard
MODE_SWITCH_KEY_KEYBOARD = 'm'
gesture_mode_active = True  # Start with gesture mode active

# Variables for gesture mode switch detection
gesture_switch_detection = []

# Main loop to switch between modes (gesture and keyboard)
while True:
    if gesture_mode_active:
        # Read a frame from the webcam for gesture recognition
        success_gesture, img_gesture = cap_gesture.read()
        if not success_gesture:
            print("Failed to capture gesture image")
            break

        # Find hands in the image
        hands, img_gesture = detector.findHands(img_gesture, draw=True)

        # Initialize lists for storing key points coordinates
        left_hand_coords = [(0, 0)] * num_landmarks
        right_hand_coords = [(0, 0)] * num_landmarks

        # Check if any hands are detected
        if hands:
            # Reset no_hand_frames count
            no_hand_frames = 0
            # Loop through each detected hand
            for hand in hands:
                lmList = hand["lmList"]  # List of 21 landmarks for the current hand
                handType = hand["type"]  # Type of the current hand ("Left" or "Right")

                if handType == "Left":
                    left_hand_coords = [(lm[0], lm[1]) for lm in lmList]
                elif handType == "Right":
                    right_hand_coords = [(lm[0], lm[1]) for lm in lmList]
        else:
            # Increment no_hand_frames count
            no_hand_frames += 1

        left_hand_angles = [0.0] * (num_landmarks - 2)
        right_hand_angles = [0.0] * (num_landmarks - 2)

        if left_hand_coords[0] != (0, 0):  # If left hand is detected
            for i in range(1, num_landmarks - 1):
                A = left_hand_coords[i - 1]
                B = left_hand_coords[i]
                C = left_hand_coords[i + 1]
                angle = calculate_angle(A, B, C)
                left_hand_angles[i - 1] = angle

        if right_hand_coords[0] != (0, 0):
           for i in range(1, num_landmarks - 1):
                A = right_hand_coords[i - 1]
                B = right_hand_coords[i]
                C = right_hand_coords[i + 1]
                angle = calculate_angle(A, B, C)
                right_hand_angles[i - 1] = angle

        # Combine the coordinates into a single list
        angles = left_hand_angles + right_hand_angles
        combined_coords = left_hand_coords + right_hand_coords

        # Calculate distances and angles between key points for gesture recognition
        features = calculate_features(combined_coords)
        features = features + angles
        # Convert features to DataFrame for prediction
        features_df = pd.DataFrame([features])

        # Make a prediction if features are available
        if not features_df.empty and hands:
            probabilities = clf.predict_proba(features_df)
            max_prob = np.max(probabilities)
            if max_prob >= confidence_threshold:
                prediction = clf.predict(features_df)
                predicted_gesture = label_map.get(prediction[0], 'Unknown')

                # Add to sentence if the gesture is different from the previous one
                if predicted_gesture != previous_gesture:
                    recognition_count = 1  # Reset the recognition
                    previous_gesture = predicted_gesture
                else:
                    recognition_count += 1

                # If the gesture is recognized twice consecutively and it is different from the last character in the sentence, add it to the sentence
                if (recognition_count == 1 and (len(sentence) == 0 or predicted_gesture != sentence[-1])) or (recognition_count == 15 and len(sentence) > 0 and predicted_gesture == sentence[-1]):
                    sentence += predicted_gesture
                    recognition_count = 0  # Reset recognition count for the next gesture
                    time.sleep(0.5)  # Wait for 0.5 seconds before recognizing the next gesture

                    # Check if "AAAA" gesture is recognized to switch modes
                    gesture_switch_detection.append(predicted_gesture)
                    if len(gesture_switch_detection) > 4:  # Check for "AAAA"
                        gesture_switch_detection.pop(0)  # Remove oldest entry
                        if all(gesture == 'A' for gesture in gesture_switch_detection):
                            gesture_mode_active = not gesture_mode_active  # Switch mode
                            print(f"Switched to {'Gesture' if gesture_mode_active else 'Keyboard'} Mode")
                            sentence = ""  # Clear sentence after mode switch
                            finalText = ""  # Clear finalText after mode switch

            else:
                previous_gesture = ""
                recognition_count = 0
        else:
            # Handle case when no hands are detected
            if no_hand_frames > 30:  # Adjust this threshold as needed
                if sentence and sentence[-1] != ' ':
                    sentence += ' '
                previous_gesture = ""
                recognition_count = 0

        # Display the gesture recognition result on the image
        cv2.putText(img_gesture, f'Gesture Mode - Sentence: {sentence.strip()}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Display the gesture recognition image
        cv2.imshow("Hand Gesture Recognition and Keyboard", img_gesture)

    else:
        # Read a frame from the webcam for keyboard input
        success_keyboard, img_keyboard = cap_keyboard.read()
        if not success_keyboard:
            print("Failed to capture keyboard image")
            break

        # Find hands in the image for keyboard interaction
        hands, img_keyboard = detector.findHands(img_keyboard, draw=True, flipType=True)
        img_keyboard = drawAll(img_keyboard, buttonList)

        if hands:
            lmListHand = hands[0]['lmList']
            for button in buttonList:
                x, y = button.pos
                w, h = button.size

                if x < lmListHand[8][0] < x + w and y < lmListHand[8][1] < y + h:
                    if currentButton != button.text:
                        startStableTime = time()
                        currentButton = button.text

                    if currentButton == button.text:
                        if startStableTime is not None and time() - startStableTime > stableTime:
                            cv2.rectangle(img_keyboard, (x - 5, y - 5), (x + w + 5, y + h + 5), (130, 80, 70), cv2.FILLED)
                            cv2.putText(img_keyboard, button.text, (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 2, (200, 200, 200), 2)

                            l, _, _ = detector.findDistance((lmListHand[8][0], lmListHand[8][1]),
                                                            (lmListHand[12][0], lmListHand[12][1]), img_keyboard)

                            currentTime = time()
                            if l < 30 and (lastKeyPressed != button.text or (currentTime - lastPressTime) > pressCooldown):
                                cv2.rectangle(img_keyboard, button.pos, (x + w, y + h), (230,50,230), cv2.FILLED)
                                cv2.putText(img_keyboard, button.text, (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
                                if button.text == "<":  # Handle backspace
                                    finalText = finalText[:-1]
                                elif button.text == "Space":  # Handle space
                                    finalText += " "
                                elif button.text == "Enter":  # Handle enter
                                    print(f"Keyboard Mode - Text Entered: {finalText}")
                                    finalText = ""  # Clear the text after printing
                                elif button.text == MODE_SWITCH_KEY_KEYBOARD:  # Handle mode switch key in keyboard mode
                                    gesture_mode_active = not gesture_mode_active  # Toggle mode
                                    print(f"Switched to {'Gesture' if gesture_mode_active else 'Keyboard'} Mode")
                                    sentence = ""  # Clear sentence after mode switch
                                    finalText = ""  # Clear finalText after mode switch
                                else:
                                    finalText += button.text
                                lastKeyPressed = button.text
                                lastPressTime = currentTime
                                startStableTime = None  # Reset stable time after a key press

        else:
            currentButton = None  # Reset currentButton when no hands are detected

        # Display the keyboard interaction result on the image
        cv2.putText(img_keyboard, f'Keyboard Mode - Text: {finalText}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Display the keyboard interaction image
        cv2.imshow("Hand Gesture Recognition and Keyboard", img_keyboard)

    # Check for mode switching key press ('g' for gesture mode, 'k' for keyboard mode)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('g'):
        gesture_mode_active = True
        print("Switched to Gesture Mode")
    elif key == ord('k'):
        gesture_mode_active = False
        print("Switched to Keyboard Mode")

    # Break the loop if 'q' is pressed
    if key == ord('q'):
        break

# Release video captures and close windows
cap_gesture.release()
cap_keyboard.release()
cv2.destroyAllWindows()

# Print the final sentence or text entered
print("Final Sentence:", sentence.strip())
print("Final Text Entered:", finalText)

Switched to Keyboard Mode


TypeError: 'module' object is not callable